In [5]:
import requests
import json



import matplotlib.pyplot as plt
import seaborn as sns

from pandasdmx import Request
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
import numpy as np
import geopandas as gpd

import time

import csv

import sys, os
from sqlalchemy import create_engine

import pymssql
import sqlalchemy as db

from functools import reduce

import folium # map rendering library
from sklearn.neighbors import NearestNeighbors



/home/holborn/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


### Here API

In [159]:
API_Key = 't9X9EF_AkG_qVZ8Q4dWSgyJuMhSklDP7i_gqX5EMunQ'
URL = 'https://router.hereapi.com/v8/routes?'
# TRANSPORTMODE ='transportMode=pedestrian' 
PARAMS = '&origin=-37.8046041,144.9454422&destination=-37.8102306,144.9539645&return=polyline,summary&apikey='

TRANSPORTMODE ='pedestrian' 

RETURN = 'polyline'

DESTINATION_X = '-37.8102306'
DESTINATION_Y = '144.9539645'
ORIGIN_X = '-37.8046041'
ORIGIN_Y = '144.9454422'

### Get Travel Time

In [160]:
def get_transport(TRANSPORTMODE, ORIGIN_X,ORIGIN_Y,DESTINATION_X,DESTINATION_Y,RETURN ,API_Key):
    
    #print(nb)
    
    #form the request url and request only the items (which are the venues)
    url = 'https://router.hereapi.com/v8/routes?transportMode={}&origin={},{}&destination={},{}&return={},summary&apikey={}'.format(TRANSPORTMODE, ORIGIN_X,ORIGIN_Y,DESTINATION_X,DESTINATION_Y,RETURN ,API_Key)
    items = requests.get(url).json()['routes'][0]['sections']
    transport = json_normalize(items).set_index('id')
    print(url)
    #include exception handling where Foursquare request fails for a particular Neighbourhood
    if items == []:
        print(" -- HERO request for ",nb,"returned no results --")
        return None
    return transport


### FourSquare API

In [161]:
#intialise Foursquare API credentials
url = 'https://api.foursquare.com/v2/venues/explore'


CLIENT_ID = 'ZY2UKTXC3KLI2DSJYFZPC3CPXMU4SGKILQOKDZK5FAMGXB2C' # your Foursquare ID
CLIENT_SECRET = '0D0TWC2MD2NDOHUUXRAYAS1DRQRAA2E14Z33QU01YADTE0O1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


# #intialise Foursquare API credentials
# CLIENT_ID = 'EKC0OWGJC1SY1AE1UHB4PUPH2JGARZTQK1U5C1USTUNA43JF' # your Foursquare ID
# CLIENT_SECRET = 'CHH0EPUEN2PH4WBDV4XHTDT5NWTUZ2SYVIQALUOZXWYNABRO' # your Foursquare Secret

### Postcodes

In [163]:
postcodes = pd.read_csv('Datasets/Australian_Post_Codes_Lat_Lon.csv')
postcodes.drop_duplicates(subset='suburb', inplace=True)
postcodes.drop(['dc', 'type'],axis=1,inplace=True )
postcodes.dropna(inplace=True) #drop the trailing row which has no data
postcodes.reset_index(inplace=True, drop=True) #reinex after having removed some rows above
postcodes.shape
postcodes.head()

,postcode,suburb,state,lat,lon
0,200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,-35.277272,149.117136
1,221,BARTON,ACT,-35.201372,149.095065
2,800,DARWIN,NT,-12.801028,130.955789
3,804,PARAP,NT,-12.432181,130.843310
4,810,ALAWA,NT,-12.378451,130.877014


In [166]:
melb_postcodes = postcodes[(postcodes['postcode']>=3000)&(postcodes['postcode']<=3207)|(postcodes['postcode']>=8000)&(postcodes['postcode']<=8499)]
melb_postcodes.head()

,postcode,suburb,state,lat,lon
4901,3000,MELBOURNE,VIC,-37.814563,144.970267
4902,3002,EAST MELBOURNE,VIC,-37.816640,144.987811
4903,3003,WEST MELBOURNE,VIC,-37.806255,144.941123
4904,3005,WORLD TRADE CENTRE,VIC,-37.822262,144.954856
4905,3006,SOUTHBANK,VIC,-37.823258,144.965926


### Get Venues

In [162]:
#Use the function to extract the category from the dataframe (because the column name could be either 'categories' or 'venue.categories')
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


#define a function to return all the venues for a given neighborhood

#pass the neighborhood (nb), the latitude (la), the radius and the limit 
def get_venues(nb, la, lo, radius, limit):
    
    #print(nb)
    
    #form the request url and request only the items (which are the venues)
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, la, lo, VERSION, limit)
    items = requests.get(url).json()["response"]['groups'][0]['items']
    
    #include exception handling where Foursquare request fails for a particular Neighbourhood
    if items == []:
        print(" -- Foursquare request for ",nb,"returned no results --")
        return None
    else:
        # flatten JSON, filter for only wanted columns then use the get_category_type funtion to replace the category list with just the category.
        venues1 = json_normalize(items)
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        venues1 =venues1.loc[:, filtered_columns]
        venues1['venue.categories'] = venues1.apply(get_category_type, axis=1)

        #define a new dataframe with the Neighbourhood information
        venues2 = pd.DataFrame(columns=['Suburb','Suburb Latitude', 
                      'Suburb Longitude']) 

        #for each of the venues returned, add the venue dataframe to the (empty) neighbourhood dataframe and fill all rows in the the neighbourhood columns with the neighborhood name, the neighbourhood latitude and longitude
        i=0
        for row in venues1:
            venues2[row]=venues1[row]
            venues2['Suburb']=nb
            venues2['Suburb Latitude']=la
            venues2['Suburb Longitude']=lo
            i=i+1

        #rename the columns
        venues2.rename(index=str,columns={"venue.name":"Venue","venue.categories":"Venue Category","venue.location.lat":"Venue Latitude","venue.location.lng":"Venue Longitude"}, inplace=True)
    
    return venues2

### Call API

In [158]:
transport  = get_transport(TRANSPORTMODE, ORIGIN_X,ORIGIN_Y,DESTINATION_X,DESTINATION_Y,RETURN ,API_Key)
transport

https://router.hereapi.com/v8/routes?transportMode=pedestrian&origin=-37.8046041,144.9454422&destination=-37.8102306,144.9539645&return=polyline,summary&apikey=t9X9EF_AkG_qVZ8Q4dWSgyJuMhSklDP7i_gqX5EMunQ


/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,type,polyline,departure.time,departure.place.type,departure.place.location.lat,departure.place.location.lng,departure.place.originalLocation.lat,departure.place.originalLocation.lng,arrival.time,arrival.place.type,arrival.place.location.lat,arrival.place.location.lng,arrival.place.originalLocation.lat,arrival.place.originalLocation.lng,summary.duration,summary.length,summary.baseDuration,transport.mode
id,,,,,,,,,,,,,,,,,,
efb71811-c925-4358-aae4-781073f8ddc6,pedestrian,BGtotjoCux4u0I5DyoBrEkwBvC0enB4STgFTsEnBsEnB4D...,2020-12-20T23:51:19+11:00,place,-37.804679,144.945431,-37.804604,144.945442,2020-12-21T00:08:45+11:00,place,-37.810234,144.953962,-37.810231,144.953965,1046,1039,1046,pedestrian


#### HERE API Call for postcode

In [164]:
#Iterate thorough all the suburbs of the dataframe df, adding each set of venues to the df_suburb_venues dataframe
#df=melb_postcodes
df=postcodes
df_suburb_venues=pd.DataFrame()
radius = 500
limit = 10000
print("Getting venue recommendations, please wait......")
for i, row in enumerate(df['suburb']):
        df_suburb_venues=df_suburb_venues.append(get_venues(df.iloc[i]['suburb'], df.iloc[i]['lat'], df.iloc[i]['lon'], radius, limit),ignore_index=True)
print("...... Complete")

Getting venue recommendations, please wait......


/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


 -- Foursquare request for  DALY RIVER returned no results --
 -- Foursquare request for  FLEMING returned no results --
 -- Foursquare request for  NGUIU returned no results --
 -- Foursquare request for  LAJAMANU returned no results --
 -- Foursquare request for  WAVE HILL returned no results --
 -- Foursquare request for  BORROLOOLA returned no results --
 -- Foursquare request for  BRUNCHILLY returned no results --
 -- Foursquare request for  AVON DOWNS returned no results --
 -- Foursquare request for  CRESSWELL DOWNS returned no results --
 -- Foursquare request for  MUCKATY STATION returned no results --
 -- Foursquare request for  NEWCASTLE WATERS returned no results --
 -- Foursquare request for  WOLLOGORANG STATION returned no results --
 -- Foursquare request for  AMATA returned no results --
 -- Foursquare request for  GIBSON DESERT NORTH returned no results --
 -- Foursquare request for  SANTA TERESA returned no results --
 -- Foursquare request for  WILLOWRA returned no r

KeyboardInterrupt: 

In [167]:
venue_property = "Datasets/venue_datasets_largest.csv"
df_suburb_venues = pd.read_csv(venue_property)
df_suburb_venues['Venue Category'].unique()

array(['Speakeasy', 'Boutique', 'Peruvian Restaurant', 'Bakery',
       'Vietnamese Restaurant', 'Asian Restaurant', 'Hotel', 'Whisky Bar',
       'Street Art', 'Italian Restaurant', 'Bookstore',
       'French Restaurant', 'Café', 'Sushi Restaurant',
       'Australian Restaurant', 'Indian Restaurant', 'Coffee Shop', 'Bar',
       'Wine Bar', 'Theater', 'BBQ Joint', 'Burger Joint',
       'Armenian Restaurant', 'Argentinian Restaurant', 'Lounge',
       'Spanish Restaurant', 'Tapas Restaurant', 'Japanese Restaurant',
       'Cocktail Bar', 'Museum', 'Steakhouse', 'Art Gallery',
       'Movie Theater', 'Juice Bar', 'Hotel Bar', 'Plaza', 'Creperie',
       'Comic Shop', 'Mexican Restaurant', 'Soba Restaurant',
       'Deli / Bodega', 'Chocolate Shop', 'Dumpling Restaurant',
       'Sandwich Place', 'Performing Arts Venue', 'Ice Cream Shop',
       'Road', 'Shanghai Restaurant', 'Thai Restaurant', 'Park',
       'Ramen Restaurant', 'Chinese Restaurant', 'Cricket Ground',
       'Breakfas

In [168]:
df_suburb_venues

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,MELBOURNE,-37.814563,144.970267,Eau de Vie,Speakeasy,-37.815748,144.971542
1,MELBOURNE,-37.814563,144.970267,Louis Vuitton,Boutique,-37.814842,144.969268
2,MELBOURNE,-37.814563,144.970267,Pastuso,Peruvian Restaurant,-37.815354,144.971062
3,MELBOURNE,-37.814563,144.970267,Lune Croissanterie,Bakery,-37.815514,144.969415
4,MELBOURNE,-37.814563,144.970267,Coda,Vietnamese Restaurant,-37.815625,144.969906
...,...,...,...,...,...,...,...
37635,LITTLE LONSDALE STREET,-37.811301,144.961819,Queen Victoria Market,Market,-37.806899,144.957954
37636,LITTLE LONSDALE STREET,-37.811301,144.961819,Wunderkammer,Antique Shop,-37.813137,144.959388
37637,LITTLE LONSDALE STREET,-37.811301,144.961819,Grill'd,Burger Joint,-37.810722,144.965694
37638,LITTLE LONSDALE STREET,-37.811301,144.961819,Gewürzhaus,Gourmet Shop,-37.815924,144.964628


In [165]:
postcodes

,postcode,suburb,state,lat,lon
0,200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,-35.277272,149.117136
1,221,BARTON,ACT,-35.201372,149.095065
2,800,DARWIN,NT,-12.801028,130.955789
3,804,PARAP,NT,-12.432181,130.843310
4,810,ALAWA,NT,-12.378451,130.877014
...,...,...,...,...,...
14264,8004,ST KILDA ROAD,VIC,-37.836219,144.975549
14265,8006,ABECKETT STREET,VIC,-37.809696,144.959314
14266,8009,FLINDERS LANE,VIC,-37.817201,144.964531
14267,8010,LAW COURTS,VIC,-38.185857,146.293728
